In [1]:
using LinearAlgebra
using ForwardDiff

In [131]:
r_rx = reshape([4.293827035231330  -2.932803253236232   0.049478811607765  -4.510052883266811 -2.713585854152024   0.288750584827538   1.775897028559260   1.143011600958173 0.016188456663874  -1.227388481747143   1.740752145645313  -0.025787452759501], (3,4))

3×4 Matrix{Float64}:
  4.29383    -4.51005   1.7759     -1.22739
 -2.9328     -2.71359   1.14301     1.74075
  0.0494788   0.288751  0.0161885  -0.0257875

In [132]:
tag_elev = 0.0

0.0

In [134]:
t_rx = 1e5*[0  -5.271400129363028  -4.172568959466880   0.480240310983936]'

4×1 Matrix{Float64}:
       0.0
 -527140.0129363028
 -417256.895946688
   48024.0310983936

In [135]:
t_offset = 1e5*[-5.271395147390924  -4.172574299864024   0.480279094964825]'

3×1 Matrix{Float64}:
 -527139.5147390924
 -417257.42998640245
   48027.9094964825

In [136]:
θ = [r_rx[:]; tag_elev; t_rx; t_offset]

20×1 Matrix{Float64}:
       4.29382703523133
      -2.932803253236232
       0.049478811607765
      -4.510052883266811
      -2.713585854152024
       0.288750584827538
       1.77589702855926
       1.143011600958173
       0.016188456663874
      -1.227388481747143
       1.740752145645313
      -0.025787452759501
       0.0
       0.0
 -527140.0129363028
 -417256.895946688
   48024.0310983936
 -527139.5147390924
 -417257.42998640245
   48027.9094964825

In [137]:
x = [-1.469209276376144   1.779965499033907  -2.330889985309173]'

3×1 adjoint(::Matrix{Float64}) with eltype Float64:
 -1.469209276376144
  1.779965499033907
 -2.330889985309173

In [198]:
function residual(x,θ)
    c = 299792458.0/1e8
    r_tag = x[1:2]
    t_tx = x[3]
    
    r_rx = reshape(θ[1:12], (3,4))
    tag_elev = θ[13]
    t_rx = θ[14:17]
    t_offset = θ[18:20]
    
    r = zeros(eltype(θ), 4)
    r[1] = norm([r_tag; tag_elev] - r_rx[:,1])/c - t_rx[1] + t_tx
    for k = 2:4
        r[k] = norm([r_tag; tag_elev] - r_rx[:,k])/c - t_rx[k] + t_offset[k-1] + t_tx
    end
    
    return r
end

residual (generic function with 1 method)

In [199]:
function jacobian(x,θ)
    c = 299792458.0/1e8
    r_tag = x[1:2]
    t_tx = x[3]
    
    r_rx = reshape(θ[1:12], (3,4))
    tag_elev = θ[13]
    t_rx = θ[14:17]
    t_offset = θ[18:20]
    
    J = zeros(eltype(θ), 4,3)
    for k = 1:4
        J[k,:] = [(r_tag - r_rx[1:2,k])'/(c*norm([r_tag; tag_elev] - r_rx[:,k])) 1];
    end
    
    return J
end

jacobian (generic function with 1 method)

In [200]:
function f(x,θ)
    return jacobian(x,θ)'*residual(x,θ)
end

f (generic function with 1 method)

In [201]:
Jθ = ForwardDiff.jacobian(dθ->f(x,dθ),θ)

3×20 Matrix{Float64}:
 -0.0694056   0.0511779  -0.00053732   …  0.186709  -0.327314   -0.327442
  0.0511779  -0.0486827   0.000439437     0.275845   0.0642493   0.0531191
  0.258203   -0.211166    0.00221705      1.0        1.0         1.0

In [196]:
H = ForwardDiff.jacobian(dx->f(dx,θ),x)

3×3 Matrix{Float64}:
  0.391636  0.274691  -0.72625
  0.274691  2.11624    0.60438
 -0.72625   0.60438    4.0

In [202]:
dxdθ = H\Jθ

3×20 Matrix{Float64}:
 -0.132953    0.0857785  -0.000827737  …   1.84474   -0.69201    -0.681382
  0.0312481  -0.0245698   0.000208726     -0.288609   0.0884871   0.0809726
  0.0356901  -0.0335051   0.000372439      0.628542   0.110987    0.114052

In [391]:
Σθ = Diagonal([kron(ones(4),[1.0/100; 1.0/100; 10.0/100]); 10.0/100; kron(ones(7),100e-9/1e-6)])

20×20 Diagonal{Float64, Vector{Float64}}:
 0.01   ⋅     ⋅    ⋅     ⋅     ⋅    ⋅    …   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    0.01   ⋅    ⋅     ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅    0.1   ⋅     ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅   0.01   ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅    0.01   ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅    0.1   ⋅    …   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅     ⋅   0.01      ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅     ⋅    ⋅    …   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅     ⋅     ⋅    ⋅        ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅     ⋅     ⋅    ⋅  

In [392]:
Σx = dxdθ*Σθ*dxdθ'

3×3 Matrix{Float64}:
  1.19651   -0.201583    0.260872
 -0.201583   0.0468957  -0.0469506
  0.260872  -0.0469506   0.101382

In [393]:
100*sqrt((Σx[1,1]+Σx[2,2]))

111.50791965013809

In [300]:
function residual2(x,θ)
    c = 299792458.0/1e8
    r_tag = x[1:3]
    t_tx = x[4]
    
    r_rx = reshape(θ[1:12], (3,4))
    t_rx = θ[14:17]
    t_offset = θ[18:20]
    
    r = zeros(eltype(θ), 4)
    r[1] = norm(r_tag - r_rx[:,1])/c - t_rx[1] + t_tx
    for k = 2:4
        r[k] = norm(r_tag - r_rx[:,k])/c - t_rx[k] + t_offset[k-1] + t_tx
    end
    
    return r
end

residual2 (generic function with 1 method)

In [301]:
function jacobian2(x,θ)
    c = 299792458.0/1e8
    r_tag = x[1:3]
    t_tx = x[4]
    
    r_rx = reshape(θ[1:12], (3,4))
    t_rx = θ[14:17]
    t_offset = θ[18:20]
    
    J = zeros(eltype(θ), 4,4)
    for k = 1:4
        J[k,:] = [(r_tag - r_rx[:,k])'/(c*norm(r_tag - r_rx[:,k])) 1];
    end
    
    return J
end

jacobian2 (generic function with 1 method)

In [302]:
x2 = [x[1:2]; 0; x[3]]

4-element Vector{Float64}:
 -1.4685997782790325
  1.779882554294614
  0.0
 -2.3307666947977963

In [296]:
residual2(x2,θ)

4-element Vector{Float64}:
  0.1523803759492206
 -0.020082980455492727
 -1.7618926700117412
  1.629595274517476

In [297]:
residual(x,θ)

4-element Vector{Float64}:
  0.1523803759492206
 -0.020082980455492727
 -1.7618926700117412
  1.629595274517476

In [299]:
ForwardDiff.jacobian(dx->residual2(dx,θ),x2)-jacobian2(x2,θ)

4×4 Matrix{Float64}:
  0.0          -2.77556e-17   0.0          0.0
  0.0           0.0           3.46945e-18  0.0
 -5.55112e-17   1.38778e-17  -4.33681e-19  0.0
  0.0           0.0           0.0          0.0

In [303]:
dxdθ2 = jacobian2(x2,θ)\ForwardDiff.jacobian(dθ->residual2(x2,dθ),θ)

4×20 Matrix{Float64}:
 -0.873041   0.713999  -0.00749632  …   2.40029    -0.0194485   1.00037
  2.17077   -1.77532    0.0186392      -1.13045    -6.91424    -0.36253
  0.656392  -0.536817   0.00563608     -0.335043  -29.3936     27.1865
 -0.424157   0.346888  -0.003642        0.857733    1.38987     0.395127

In [322]:
Σθ2 = Diagonal([kron(ones(4),[1.0/100; 1.0/100; 3.0/100]); 0.0/100; kron(ones(7),30e-9/1e-6)])

20×20 Diagonal{Float64, Vector{Float64}}:
 0.01   ⋅     ⋅     ⋅     ⋅     ⋅    …   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅    0.01   ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅    0.03   ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅    0.01   ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅    0.01   ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅    0.03  …   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅    …   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    

In [323]:
Σx2 = dxdθ2*Σθ2*dxdθ2'

4×4 Matrix{Float64}:
  0.769037  -1.06422   1.38107   0.321159
 -1.06422    5.20682  12.5057   -1.08499
  1.38107   12.5057   98.1949   -1.9874
  0.321159  -1.08499  -1.9874    0.256523

In [324]:
100*sqrt(tr(Σx2[1:3,1:3]))

1020.6405482145249